# Necessary packages and functions

In [1]:
#pop-up windows for figures:
# %matplotlib qt
#nice in-line figures:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import scipy as sp
from scipy.stats import multivariate_normal
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D 
import random
import sys
from scipy import linalg
from scipy.signal import savgol_filter
from matplotlib.pyplot import savefig
import matplotlib as mpl

sigmoid = lambda x,c,a: 1/(1 + np.exp(-a*(x-c))) #reward sensitivities

# MP Linear Chain (Figure 3)

Parameters

In [2]:
num_states=4 #number of timesteps in the MP
alpha=0.01 #learning rate 
num_gammas=100 #number of gammas

# Use gammas equally separated in the 1/log(gamma) space:
taus=np.linspace(0.01,3,num_gammas)
range_g=np.exp(-1/taus)

# Use gammas equally separated in the linear space:
# range_g=np.linspace(0.0,1,num_gammas)


num_trials=1000 #number of trials used to learn the TD code

width=13 #steepness of the reward sensitivities
num_h=250 #number of h's
range_h=np.linspace(-3,3,num_h) #theta_h range

TD backups

In [3]:
i_g=0 #counter for the gammas
V_gamma=np.zeros((len(range_g),num_states,num_h)) #initialize gamma-space

for gamma in range_g[0:-1]: #loop over gammas
    i_g=i_g+1
    V=np.zeros((num_states,num_h))
    for t in range(1,num_trials): #loop over trials
        for s in range(1,num_states+1): #walk along the chain
            
            #Specify the immediate rewards at each timestep:
            if s==1:
                state=1
                r1=np.random.choice([-2,2]);
            elif s==2:
                state=2
                r2=np.random.choice([-2,2])
            elif s==3:
                state=3
                r3=np.random.choice([-1,1])
            elif s==4:
                state=4
                r4=np.random.choice([-1,1])
            
            
            #Implement the 2-dimensional TD backups:
            if state==1:
                    R=sigmoid(r1,range_h,width); #R=f_h(r1)
                    delta= R + gamma*V[1,:] - V[0,:]
                    V[0,:]=V[0,:]+alpha*delta;
            elif state==2:
                    R=sigmoid(r2,range_h,width); 
                    delta= R + gamma*V[2,:] - V[1,:];
                    V[1,:]=V[1,:]+alpha*delta;   
            elif state==3:
                    R=sigmoid(r3,range_h,width);
                    delta= R + gamma*V[3,:] - V[2,:];
                    V[2,:]=V[2,:]+alpha*delta;  
            elif state==4:
                    R=sigmoid(r4,range_h,width);
                    delta= R - V[3,:];
                    V[3,:]=V[3,:]+alpha*delta;
    V_gamma[i_g,:,:]=V #Add learned gamma to gamm-space
    print(i_g,end=' ')


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

### $\gamma$-Space

In [4]:
#Set up gamma-space:
V_gamma2=V_gamma[4:-1,0,:]
Z=V_gamma2[:,0:-2]-V_gamma2[:,1:-1]

In [5]:
#Plot figure:
X,Y=np.meshgrid(range_h[0:-2],range_g[4:-1]);

for h in range(0,num_h-2):
    Z[:,h]=savgol_filter(Z[:,h], 15, 1)  

        
fig = plt.figure(dpi=150)
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap='summer'
                       , edgecolor='none',alpha=1)
ax.view_init(60, -45)
# ax.set_zlim(0,0.15)
ax.grid(b=None)
plt.yticks([0,0.5,1])
plt.xticks([-2,-1,0,1,2])
ax.set_zlabel('Convergence value')
ax.set_ylabel('Temporal Discount')
ax.set_xlabel('Reward')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### $\tau$-space using the SVD-based approximation to $\mathbf{L}^{-1}$

In [6]:
alpha_reg=0.2 #Regularization parameter

K=4 #Temporal horizon
delta_t=1 #Length of timestep

#define matrix F:
F=np.zeros((len(range_g),K))
for i_g in range(0,len(range_g)):
    for i_t in range(0,K):
        F[i_g,i_t]=range_g[i_g]**(i_t*delta_t)

        
U, lam, V = linalg.svd(F) #SVD decomposition of F

#set up gamma-space:
V_gamma2=V_gamma[:,0,:]
Z=V_gamma2[:,0:-2]-V_gamma2[:,1:-1]


#smooth gamma-space (it might not be necessary, it helps if the input is *very* noisy):
# for h in range(0,num_h-2):
#     Z[:,h]=savgol_filter(Z[:,h], 5, 1)

#Linearly recover tau-space from eigenspace of F:
tau_space=np.zeros((K,num_h-2))
for h in range(0,num_h-2):
    term=np.zeros((1,K))
    for i in range(0,len(lam)):
        fi=lam[i]**2/(alpha_reg**2+lam[i]**2)
        new=fi*(((U[:,i]@Z[:,h])*V[i,:] )/lam[i])
        term=term+new
    tau_space[:,h]=term

    
#smooth gamma-space (it might not be necessary, use for a smoother visualization):
# for h in range(0,num_h-2):
#     tau_space[:,h]=savgol_filter(tau_space[:,h], 11, 1)


#Normalization (it is not necessary for this very short temporal horizon T=4):
tau_space[tau_space<0]=0 #make all probabilities positive
for i in range(0,len(tau_space)): #normalize
    if np.nansum(tau_space[i,:])>0.0:
        tau_space[i,:]=tau_space[i,:]/np.nansum(tau_space[i,:])
        


In [7]:
#Plot figure
plt.close(); fig=plt.figure()
plt.clf()


ax = fig.gca(projection='3d')
X,Y=np.meshgrid(range_h[0:-2],delta_t*np.linspace(0,K-1,K)) #grid to plot

surf = ax.plot_surface(X, Y, tau_space, cmap='summer'
                           , edgecolor='none',alpha=1)


ax.grid(b=None)

ax.view_init(60, -45)
plt.yticks([0,1,2,3])
plt.xticks([-2,-1,0,1,2])
ax.set_zlim(0,0.085)
ax.set_zlabel('Probability')
ax.set_ylabel('Future Time')
ax.set_xlabel('Reward')
fig.canvas.draw_idle()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# MDP Double Chain (Figure 5)

Parameters

In [19]:
num_states=6 #number of timesteps in the MP
alpha=0.01 #learning rate 
num_gammas=100 #number of gammas

# Use gammas equally separated in the linear space:
# range_g=np.linspace(0,1,num_gammas)

# Use gammas equally separated in the 1/log(gamma) space:
taus=np.linspace(0.01,num_states,num_gammas)
range_g=np.exp(-1/taus)

num_trials=3000  #number of trials used to learn the TD code
width=25 #steepness of the reward sensitivities
num_h=150 #number of h's
range_h=np.linspace(-0.25,1.25,num_h) #theta_h range

TD backups

In [20]:
i_g=0 #counter for the gammas
V_gamma=np.zeros((len(range_g),2*num_states-1,num_h)) #initialize V_{h,\gamma}

for gamma in range_g[0:-2]:
    i_g=i_g+1

    V=np.zeros((2*num_states-1,num_h))    #set V such that components [0:num_states-1] belong to branch 1 (top branch in Figure 5), 
   #and components [num_states:2*num_states-1] to branch 2 (bottom in Figure 5)
    for t in range(1,num_trials):
        for t in range(1,num_states+1): 

            
            # The reward (r) at each timestep t will depend on which branch you are:
            if t==1:
                r=0;
                branch=1+np.random.binomial(1, 0.5) # Choose branch
            elif t==2 and branch==1:
                r=0.6
            elif t==2 and branch==2:
                r=0
            elif t==3:
                r=0
            elif t==4:
                r=0
            elif t==5:
                r=0
            elif t==6 and branch==1:
                r=0
            elif t==6 and branch==2:
                r=1 
                
            
            R=sigmoid(r,range_h,width) #Apply reward sensitivity to the reward r
            
            if t==1 and branch==1: 
                delta= R + gamma*V[t,:] - V[t-1,:] #In branch=1 the transition is from state 0 to state 1
                V[t-1,:]=V[t-1,:]+alpha*delta 
            elif t==1 and branch==2:
                delta= R + gamma*V[t+num_states,:] - V[t-1,:] #In branch=2 the transition is from state 0 to state num_states+1
                V[t-1,:]=V[t-1,:]+alpha*delta 
                
            elif t<num_states and branch==1:
                delta= R + gamma*V[t,:] - V[t-1,:] #Branch 1 states go from 1 to num_states
                V[t-1,:]=V[t-1,:]+alpha*delta
            elif t==num_states and branch==1:
                delta= R  - V[t-1,:]
                V[t-1,:]=V[t-1,:]+alpha*delta
                
            elif t<num_states and branch==2:
                delta= R + gamma*V[t+num_states-1,:] - V[t+num_states-2,:]  #Branch 2 states go from num_states to 2*num_sates-2
                V[t+num_states-2,:]=V[t+num_states-2,:]+alpha*delta
            elif t==num_states and branch==2:
                delta= R  - V[t+num_states-2,:]
                V[t+num_states-2,:]=V[t+num_states-2,:]+alpha*delta

    V_gamma[i_g,:,:]=V
    print(i_g,end=' ')


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 

### $\gamma$-space

In [21]:
#Set up gamma-space:
V_gamma2=V_gamma[:,0,:]
Z=V_gamma2[:,0:-2]-V_gamma2[:,1:-1]

In [22]:
#Plot Figure
fig = plt.figure()
ax = fig.gca(projection='3d')
X,Y=np.meshgrid(range_h[0:-2],range_g);

# for h in range(0,num_h-2):
#     Z[:,h]=savgol_filter(Z[:,h], 11, 1)  

surf = ax.plot_surface(X, Y, Z, cmap='summer'
                       , edgecolor='none',vmin=0,vmax=0.07)

ax.view_init(50, -45)
ax.set_zlabel('Convergence value')
ax.set_ylabel('Temporal Discount')
ax.set_xlabel('Reward')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### $\tau$-space using the SVD-based approximation to $\mathbf{L}^{-1}$

In [24]:
alpha_reg=0.6 #regularization parameter


K=num_states-1 #Temporal horizon of tau-space
delta_t=1/4 #Temporal resolution. Delta t=0.25 is chosen here. The small Delta t helps with regularization.
# If temporal resolution is changed, alpha_reg needs to change accordingly.
K=int(K/delta_t) #K is now the total length of the tau_space

#Set up matrix F:
F=np.zeros((len(range_g),K))
for i_g in range(0,len(range_g)):
    for i_t in range(0,K):
        F[i_g,i_t]=range_g[i_g]**(i_t*delta_t)

# SVD of F:
U, lam, V = linalg.svd(F)


#set up gamma-space:
V_gamma2=V_gamma[:,0,:]
Z=V_gamma2[:,0:-2]-V_gamma2[:,1:-1]

#smooth gamma space (not necessary):
# for h in range(0,num_h-2):
#     Z[:,h]=savgol_filter(Z[:,h], 7, 1)

#Linearly recover tau-space from eigenspace of F:
tau_space=np.zeros((K,num_h-2))
for h in range(0,num_h-2):
    term=np.zeros((1,K))
    for i in range(0,len(lam)):
        fi=lam[i]**2/(alpha_reg**2+lam[i]**2)
        new=fi*(((U[:,i]@Z[:,h])*V[i,:] )/lam[i])
        term=term+new
    tau_space[:,h]=term

#Smooth tau-space (not necessary):
# for h in range(0,num_h-2):
#     tau_space[:,h]=savgol_filter(tau_space[:,h], 7, 1)


#Normalization. It is crucial when K is large (this is, if a small Delta t is chosen)
tau_space[tau_space<0]=0 #Probabilities are positive
for i in range(0,len(tau_space)): #normalize
    if np.nansum(tau_space[i,:])>0.001:
        tau_space[i,:]=tau_space[i,:]/np.nansum(tau_space[i,:])
        

In [25]:
#Plot figure
plt.close(); fig=plt.figure()

plt.clf()
ax = fig.gca(projection='3d')

X,Y=np.meshgrid(range_h[0:-2],delta_t*np.linspace(0,K,K))

surf = ax.plot_surface(X, Y, tau_space, cmap='summer'
                           , edgecolor='none')


ax.view_init(50, -45)
# plt.ylim(0, 3.1)
ax.set_zlim(0,0.04)
fig.canvas.draw_idle()
ax.set_zlabel('Probability')
ax.set_ylabel('Future Time')
ax.set_xlabel('Reward')
# savefig('demo.png', transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Reward')

# Laplace code that only varies along $\gamma$, not $h$ (Figure A.1)

Parameters

In [27]:
#set up MP:
num_states=9 #number of timesteps in the MP
position_reward_1=1 
position_reward_2=8

alpha=0.1 #learning rate 

num_gammas=100 #number of gammas

#Gammas equidistant in the 1/log space:
taus=np.linspace(0.1,num_states,num_gammas)
range_g=np.exp(-1/taus) #Choose gammas

num_trials=3000  #number of trials used to learn the TD code

TD backups

In [28]:
i_g=0 #counter for the gammas
V_gamma=np.zeros((len(range_g),num_states)) #initialize V_{h,\gamma}
for gamma in range_g[0:-1]:
    i_g=i_g+1
    V=np.zeros((num_states))
    for t in range(1,num_trials):
        for s in range(1,num_states+1):
            
            r=0
            state=s-1
            if state==position_reward_1: 
                r=1
            elif state==position_reward_2: 
                r=1
    

            end_state=num_states-1
                
            if state<end_state:
                delta= r + gamma*V[state+1] - V[state]
                V[state]=V[state]+alpha*delta;
            elif state==end_state:
                delta= r  - V[state]
                V[state]=V[state]+alpha*delta;
                
                
    V_gamma[i_g,:]=V[:] 
    print(i_g,end=' ')


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

### $\gamma$-space

In [29]:
plt.close(); fig=plt.figure()
Z=V_gamma[:,0]

plt.plot(range_g[:],Z*1.04,color='tab:blue',linewidth=4)
plt.plot(range_g[:],Z*1.04,'o',color='tab:blue',markersize=6)
plt.yticks(ticks=[0,1,2])
plt.xlabel('Temporal Discount ($\gamma$)')
plt.ylabel('Convergence Value ($V_\gamma$)')
plt.grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### $\tau$-space using the SVD-based approximation to $\mathbf{L}^{-1}$

In [30]:
alpha_reg=0 # tau-space Without regularization
# alpha_reg=0.0001 # tau-space With regularization

Z=V_gamma[1:-1,0]

K=num_states+2
delta_t=1

A=np.zeros((len(range_g[1:-1]),K))
for i_g in range(0,len(range_g[1:-1])):
    for i_t in range(0,K):
        A[i_g,i_t]=range_g[i_g]**(i_t*delta_t)
     

    
U, lam, V = linalg.svd(A)

tau_space=np.zeros((K,1))

term=np.zeros((1,K))
for i in range(0,len(lam)):
    fi=lam[i]**2/((alpha_reg)**2+lam[i]**2)
    new=fi*(((U[:,i]@Z)*V[i,:] )/lam[i])
    term=term+new
tau_space=term


plt.close(); fig=plt.figure(figsize=(6, 2))

plt.plot(delta_t*np.linspace(0,K-1,K),tau_space.T,linewidth=4,color='tab:blue')
plt.plot(delta_t*np.linspace(0,K-1,K),tau_space.T,'o',color='tab:blue',markersize=6)
plt.yticks(ticks=[0,1])
plt.xlabel('Future Time ($\\tau$)')
plt.ylabel('Expected reward $E[r_\\tau]$')
plt.grid()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Coffee example from the video presentation

Parameters

In [35]:
num_states=15 #number of states in the MP
alpha=0.01 #learning rate
num_gammas=100 #number of gammas

# the gammas are equally separated in the 1/log(gamma) space:
taus=np.linspace(0.01,num_states,num_gammas)
range_g=np.exp(-1/taus)
# range_g=np.linspace(0.0,1,num_gammas)

# range_g=np.linspace(0,1,num_gammas)

num_trials=3000 #number of trials
width=2 #width of the reward sensitivities
num_h=250 #number of h's
range_h=np.linspace(-6,8,num_h) #theta_h range

TD backups

In [36]:
i_g=0 #counter for the gammas
V_gamma=np.zeros((len(range_g),num_states,num_h)) #initialize gamma-space

for gamma in range_g[0:-1]: #loop over gammas
    i_g=i_g+1
    V=np.zeros((num_states,num_h))
    for t in range(1,num_trials): #loop over trials
        for s in range(1,num_states+1): #walk along the chain
            
            if s==1:
                r=np.random.choice([-5,-4])
            elif s==2:
                r=np.random.choice([-4,-3])
            elif s==3:
                r=np.random.choice([-3,-2])
            elif s==4:
                r=np.random.choice([-2,-1])
            elif s==5:
                r=np.random.choice([4,5,6])
            elif s==6:
                r=np.random.choice([4,5,6])
            elif s==7:
                r=np.random.choice([4,5,6])
            elif s==8:
                r=np.random.choice([4,5,6])
            elif s==9:
                r=np.random.choice([4,5,6])
            elif s==10:
                r=np.random.choice([4,5,6])
            elif s==11:
                r=np.random.choice([4,5])
            elif s==12:
                r=np.random.choice([3,4])
            elif s==13:
                r=np.random.choice([2,3])
            elif s==14:
                r=np.random.choice([1,2])
            elif s==15:
                r=np.random.choice([0,1])
            
            if s<num_states:
                R=sigmoid(r,range_h,width);
                delta= R + gamma*V[s,:] - V[s-1,:]
                V[s-1,:]=V[s-1,:]+alpha*delta
            else:
                delta= R - V[s-1,:]
                V[s-1,:]=V[s-1,:]+alpha*delta
                
    V_gamma[i_g,:,:]=V
    print(i_g,end=' ')


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

### $\gamma$-Space

In [37]:
from scipy import linalg
from scipy.signal import savgol_filter
fig = plt.figure()
ax = fig.gca(projection='3d')
V_gamma2=V_gamma[4:-1,0,:]
Z=V_gamma2[:,0:-2]-V_gamma2[:,1:-1]

X,Y=np.meshgrid(range_h[0:-2],range_g[4:-1]);


surf = ax.plot_surface(X, Y, Z, cmap='summer'
                       , edgecolor='none',alpha=1)

ax.view_init(60, -45)

ax.set_zlim(0,0.15)

ax.grid(b=None)
plt.yticks([0,0.5,1])
plt.xticks([-2,-1,0,1,2])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### $\tau$-space by approximating $\mathcal{L}^{-1}$ with SVD

In [38]:
alpha_reg=0.3 #Regularization parameter

K=14 #Temporal horizon
delta_t=1/2 #Temporal resolution
K=int(K/delta_t)

#define matrix F:
F=np.zeros((len(range_g),K))
for i_g in range(0,len(range_g)):
    for i_t in range(0,K):
        F[i_g,i_t]=range_g[i_g]**(i_t*delta_t)

        
U, lam, V = linalg.svd(F) #SVD decomposition of F

#set up gamma-space:
V_gamma2=V_gamma[:,0,:]
Z=V_gamma2[:,0:-2]-V_gamma2[:,1:-1]


#Linearly recover tau-space from eigenspace of F:
tau_space=np.zeros((K,num_h-2))
for h in range(0,num_h-2):
    term=np.zeros((1,K))
    for i in range(0,len(lam)):
        fi=lam[i]**2/(alpha_reg**2+lam[i]**2)
        new=fi*(((U[:,i]@Z[:,h])*V[i,:] )/lam[i])
        term=term+new
    tau_space[:,h]=term



#Normalization:
tau_space[tau_space<0]=0 #make all probabilities positive
for i in range(0,len(tau_space)): #normalize
    if np.nansum(tau_space[i,:])>0.0:
        tau_space[i,:]=tau_space[i,:]/np.nansum(tau_space[i,:])
        


#smooth gamma-space (it might not be necessary, use for a smoother visualization):
for h in range(0,num_h-2):
    tau_space[:,h]=savgol_filter(tau_space[:,h], 7, 1)


#Plot figure
plt.close(); fig=plt.figure()
plt.clf()
ax = fig.gca(projection='3d')

X,Y=np.meshgrid(range_h[0:-2],delta_t*np.linspace(0,K,K)) #grid to plot

surf = ax.plot_surface(X, Y, tau_space, cmap='summer'
                           , edgecolor='none',alpha=1)
# ax.grid(b=None)
ax.grid(b=None)
ax.view_init(70, -45)
# plt.yticks(np.linspace(0,num_states,num_states+1))
ax.set_zlabel('Probability')
ax.set_ylabel('Future Time')
ax.set_xlabel('Reward')
# ax.set_zlim(0,0.085)
fig.canvas.draw_idle()
# savefig('demo.png', transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …